In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
import imgaug.augmenters as iaa
import imgaug as ia
from sklearn.metrics import classification_report

In [ ]:
#iamge augmenation
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
class cifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, [batch_y, batch_y, batch_y, batch_y, batch_y]

In [ ]:
#callback for save model
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_model_accuracy') + logs.get(
            'val_model_1_accuracy') + logs.get(
                'val_model_2_accuracy') + logs.get(
                    'val_model_3_accuracy') + logs.get('val_model_4_accuracy')

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
#distance loss
def descriptor(X, mask = False, max_norm = False):
  X = tf.reduce_mean(X, axis=3)
  if mask:
    mean_X = tf.reduce_mean(X, axis = [1,2], keepdims= True)
    X = tf.where(X > mean_X, X , tf.zeros_like(X, dtype=tf.float32))
  if max_norm:
    max_X=tf.reduce_max(X, axis = [1,2], keepdims= True)
    X = tf.math.divide_no_nan(X, max_X)
  return X 
  
def compute_euclidean_sum(X, Y):
  L2_distance = tf.reduce_mean(tf.exp(-(X - Y)**2), axis= [1,2])
  return tf.reduce_mean(L2_distance)

def compute_cosine_sum(X, Y):
  X = tf.reshape(X, [tf.shape(X)[0], -1])
  Y = tf.reshape(Y, [tf.shape(Y)[0], -1])
  loss = -keras.losses.cosine_similarity(X,Y,axis=-1)
  return tf.reduce_mean(loss)

def cosine_euclidean_sum_loss(x, cosine_weight, euclidean_weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)
  cosine_sum = compute_cosine_sum(descriptor0, descriptor1)+compute_cosine_sum(descriptor0, descriptor2)+compute_cosine_sum(descriptor0, descriptor3)+compute_cosine_sum(descriptor0, descriptor4)+compute_cosine_sum(descriptor1, descriptor2)+compute_cosine_sum(descriptor1, descriptor3)+compute_cosine_sum(descriptor1, descriptor4)+compute_cosine_sum(descriptor2, descriptor3)+compute_cosine_sum(descriptor2, descriptor4)+compute_cosine_sum(descriptor3, descriptor4)
  euclidean_sum = compute_euclidean_sum(descriptor0, descriptor1)+compute_euclidean_sum(descriptor0, descriptor2)+compute_euclidean_sum(descriptor0, descriptor3)+compute_euclidean_sum(descriptor0, descriptor4)+compute_euclidean_sum(descriptor1, descriptor2)+compute_euclidean_sum(descriptor1, descriptor3)+compute_euclidean_sum(descriptor1, descriptor4)+compute_euclidean_sum(descriptor2, descriptor3)+compute_euclidean_sum(descriptor2, descriptor4)+compute_euclidean_sum(descriptor3, descriptor4)
  return cosine_sum, euclidean_sum, cosine_weight*cosine_sum+euclidean_weight*euclidean_sum  

In [ ]:
#create single base ResNet architecture
def creat_ResNet(input_shape, num_classes=100, num_filters = 64, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x)

  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(2*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(4*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 

  '''  
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(8*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 
  '''

  x = keras.layers.GlobalAveragePooling2D()(last_acti)
  outputs = keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  model = keras.Model(inputs=inputs, outputs=[outputs,last_acti])
  return model

In [ ]:
#create five base models for training 
input_size = (32, 32, 3)
model0 = creat_ResNet(input_shape=input_size, num_classes=10, num_filters = 64, random_seed=1)
model1 = creat_ResNet(input_shape=input_size, num_classes=10, num_filters = 64, random_seed=2)
model2 = creat_ResNet(input_shape=input_size, num_classes=10, num_filters = 64, random_seed=3)
model3 = creat_ResNet(input_shape=input_size, num_classes=10, num_filters = 64, random_seed=4)
model4 = creat_ResNet(input_shape=input_size, num_classes=10, num_filters = 64, random_seed=5)
allinputs = keras.Input(shape=input_size)
output0, feature_maps0 = model0(allinputs)
output1, feature_maps1 = model1(allinputs)
output2, feature_maps2 = model2(allinputs)
output3, feature_maps3 = model3(allinputs)
output4, feature_maps4 = model4(allinputs)
model_train = keras.Model(inputs=allinputs, outputs=[output0, output1, output2, output3, output4])

#implement distance loss
loss1, loss2, loss = cosine_euclidean_sum_loss(x=[feature_maps0, feature_maps1, feature_maps2, feature_maps3, feature_maps4], 
                                      cosine_weight=1, euclidean_weight=10, mask=True, max_norm=False)
model_train.add_loss(loss)
model_train.add_metric(loss1, name="loss1", aggregation='mean')
model_train.add_metric(loss2, name="loss2", aggregation='mean')

model_train.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                    metrics=['accuracy'])

#load training, validation and testing samples
(x_origin_train, y_origin_train), (x_test,y_test) = keras.datasets.cifar10.load_data()

y_origin_train = y_origin_train.reshape(-1,)
y_test = y_test.reshape(-1,)

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_origin_train,
    y_origin_train,
    test_size=0.25,
    random_state=0,
    stratify=y_origin_train)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)
#randomly select training samples
train_index = []
random.seed(0)
for i in range(10):
    train_index += (random.sample(list(np.where(y_train == i)[0]), 100))# number of training samples per class
random.shuffle(train_index)
sampled_x_train = x_train[train_index]
sampled_y_train = y_train[train_index]
del train_index

train_sequence = cifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = cifar10data(x_val, y_val, batch_size=10, train=False)

#train base models and save parameters
history = model_train.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))
frame = pd.DataFrame({
    'loss':
    history.history['loss'],
    'loss1':
    history.history['loss1'],
    'loss2':
    history.history['loss2'],
    'model_0_loss':
    history.history['model_loss'],
    'model_1_loss':
    history.history['model_1_loss'],
    'model_2_loss':
    history.history['model_2_loss'],
    'model_3_loss':
    history.history['model_3_loss'],
    'model_4_loss':
    history.history['model_4_loss'],
    'model_0_accuracy':
    history.history['model_accuracy'],
    'model_1_accuracy':
    history.history['model_1_accuracy'],
    'model_2_accuracy':
    history.history['model_2_accuracy'],
    'model_3_accuracy':
    history.history['model_3_accuracy'],
    'model_4_accuracy':
    history.history['model_4_accuracy'],
    'val_loss':
    history.history['val_loss'],
    'val_loss1':
    history.history['val_loss1'],
    'val_loss2':
    history.history['val_loss2'],
    'val_model_0_loss':
    history.history['val_model_loss'],
    'val_model_1_loss':
    history.history['val_model_1_loss'],
    'val_model_2_loss':
    history.history['val_model_2_loss'],
    'val_model_3_loss':
    history.history['val_model_3_loss'],
    'val_model_4_loss':
    history.history['val_model_4_loss'],
    'val_model_0_accuracy':
    history.history['val_model_accuracy'],
    'val_model_1_accuracy':
    history.history['val_model_1_accuracy'],
    'val_model_2_accuracy':
    history.history['val_model_2_accuracy'],
    'val_model_3_accuracy':
    history.history['val_model_3_accuracy'],
    'val_model_4_accuracy':
    history.history['val_model_4_accuracy'],
})
frame.to_excel('table_100_samples_seed_fix.xlsx')


Epoch 1/500
3/3 [==============================] - 22s 2s/step - loss: 18.9987 - model_loss: 4.5080 - model_1_loss: 3.5821 - model_2_loss: 3.0825 - model_3_loss: 3.8878 - model_4_loss: 3.9382 - model_accuracy: 0.1000 - model_1_accuracy: 0.2333 - model_2_accuracy: 0.0667 - model_3_accuracy: 0.1667 - model_4_accuracy: 0.1000 - loss1: 7.8649 - loss2: 8.0665 - val_loss: 60.9148 - val_model_loss: 9.4348 - val_model_1_loss: 10.7208 - val_model_2_loss: 15.3369 - val_model_3_loss: 8.8086 - val_model_4_loss: 16.6137 - val_model_accuracy: 0.1000 - val_model_1_accuracy: 0.0990 - val_model_2_accuracy: 0.1005 - val_model_3_accuracy: 0.0980 - val_model_4_accuracy: 0.1000 - val_loss1: 9.3003 - val_loss2: 7.1077
Epoch 2/500
3/3 [==============================] - 3s 1s/step - loss: 9.7530 - model_loss: 2.1742 - model_1_loss: 1.8171 - model_2_loss: 1.7473 - model_3_loss: 1.8683 - model_4_loss: 2.1461 - model_accuracy: 0.3667 - model_1_accuracy: 0.3667 - model_2_accuracy: 0.4000 - model_3_accuracy: 0.366

In [ ]:
#evualate the five base models
x_test = x_test/255.0
evaluate = model_train.evaluate(x_test,[keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test)],batch_size=10)
evaluate


1000/1000 [==============================] - 16s 14ms/step - loss: 27.9986 - model_loss: 5.5433 - model_1_loss: 4.9229 - model_2_loss: 6.3514 - model_3_loss: 5.3497 - model_4_loss: 5.8314 - model_accuracy: 0.2210 - model_1_accuracy: 0.2383 - model_2_accuracy: 0.2275 - model_3_accuracy: 0.2176 - model_4_accuracy: 0.2224 - loss1: 7.6969 - loss2: 7.9395


In [ ]:
#hard voting ensemble
class Ensemble:
    def __init__(self, model, testdata ,batch_size):
        self.ensemble_model = model
        self.testdata = testdata
        self.batch_size = batch_size
        self.data_size = testdata[1].shape[0]
    def prediction(self):
        predictions =[]
        for i in range(int(self.data_size/self.batch_size)):
            batch_predictions = np.argmax(self.ensemble_model.predict(self.testdata[0][i*self.batch_size:(i+1)*self.batch_size]),axis=2) 
            batch_predictions = np.stack(batch_predictions, axis=1)
            batch_predictions = [np.argmax(np.bincount(pre)) for pre in batch_predictions]
            predictions = predictions+batch_predictions
        return np.array(predictions)

    def evaluate(self):
        predictions = self.prediction()
        accuracy = np.sum(
            predictions == self.testdata[1]) / self.data_size
        return accuracy
AEnsemble = Ensemble(model = model_train,testdata = (x_test, y_test),batch_size=10)
AEnsemble.evaluate()

0.232

In [ ]:
#soft voting ensemble
def SoftVotingEnsemble(model, x, y):
  data_size = x.shape[0]
  predictions = tf.argmax(tf.reduce_mean(model.predict(x), axis = 0),axis=1)
  accuracy = np.sum(
     predictions == y) / data_size
  return accuracy
SoftVotingEnsemble(model_train, x_test, y_test)

0.2339

In [ ]:
#callback for ensemble learning
class eCustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(eCustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        #self.model.save(self.save_path)

In [ ]:
#samples for ensemble learning 
class ecifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, batch_y

In [ ]:
#feature fusion model 
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
extractor_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].layers[42].output)
extractor_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].layers[42].output)
extractor_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].layers[42].output)
extractor_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].layers[42].output)
extractor_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].layers[42].output)
extractor_1.trainable = False
extractor_2.trainable = False
extractor_3.trainable = False
extractor_4.trainable = False
extractor_5.trainable = False
inputs = keras.Input(shape = input_size)
features_1 = extractor_1(inputs, training = False)
features_2 = extractor_2(inputs, training = False)
features_3 = extractor_3(inputs, training = False)
features_4 = extractor_4(inputs, training = False)
features_5 = extractor_5(inputs, training = False)
x = keras.layers.Concatenate(axis=-1)([features_1,features_2,features_3,features_4,features_5])
#x = keras.layers.add([features_1,features_2,features_3,features_4,features_5])
#x = keras.layers.Conv2D(1024, (1, 1), activation='relu', padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=None))(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Activation('relu')(x)
#x = keras.layers.Dropout(0.4)(x)
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128,  name='dense_100')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
train_sequence = ecifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=False)
validation_sequence = ecifar10data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
3/3 [==============================] - 6s 2s/step - loss: 2.1469 - accuracy: 0.3333 - val_loss: 15.3581 - val_accuracy: 0.2000
Epoch 2/50
3/3 [==============================] - 2s 918ms/step - loss: 0.6941 - accuracy: 0.8333 - val_loss: 11.9974 - val_accuracy: 0.1935
Epoch 3/50
3/3 [==============================] - 2s 931ms/step - loss: 0.3356 - accuracy: 0.9667 - val_loss: 9.5310 - val_accuracy: 0.1965
Epoch 4/50
3/3 [==============================] - 2s 933ms/step - loss: 0.3059 - accuracy: 0.9333 - val_loss: 8.2215 - val_accuracy: 0.1960
Epoch 5/50
3/3 [==============================] - 2s 934ms/step - loss: 0.3002 - accuracy: 1.0000 - val_loss: 7.3340 - val_accuracy: 0.1980
Epoch 6/50
3/3 [==============================] - 2s 914ms/step - loss: 0.1754 - accuracy: 1.0000 - val_loss: 6.7036 - val_accuracy: 0.1995
Epoch 7/50
3/3 [==============================] - 2s 977ms/step - loss: 0.2632 - accuracy: 0.9333 - val_loss: 6.2135 - val_accuracy: 0.2015
Epoch 8/50
3/3 [=====

In [ ]:
#fine tuning
extractor_1.trainable = True
extractor_2.trainable = True
extractor_3.trainable = True
extractor_4.trainable = True
extractor_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
3/3 [==============================] - 10s 2s/step - loss: 0.0319 - accuracy: 1.0000 - val_loss: 3.5197 - val_accuracy: 0.2190
Epoch 2/20
3/3 [==============================] - 2s 941ms/step - loss: 0.0386 - accuracy: 1.0000 - val_loss: 3.4977 - val_accuracy: 0.2150
Epoch 3/20
3/3 [==============================] - 2s 951ms/step - loss: 0.0572 - accuracy: 1.0000 - val_loss: 3.4753 - val_accuracy: 0.2150
Epoch 4/20
3/3 [==============================] - 2s 927ms/step - loss: 0.0278 - accuracy: 1.0000 - val_loss: 3.4533 - val_accuracy: 0.2130
Epoch 5/20
3/3 [==============================] - 2s 941ms/step - loss: 0.0304 - accuracy: 1.0000 - val_loss: 3.4331 - val_accuracy: 0.2135
Epoch 6/20
3/3 [==============================] - 2s 944ms/step - loss: 0.0461 - accuracy: 1.0000 - val_loss: 3.4163 - val_accuracy: 0.2155
Epoch 7/20
3/3 [==============================] - 2s 947ms/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 3.4001 - val_accuracy: 0.2150
Epoch 8/20
3/3 [======

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.1981    0.0630    0.0956      1000\n           1     0.2443    0.1710    0.2012      1000\n           2     0.1165    0.1290    0.1224      1000\n           3     0.1726    0.1690    0.1708      1000\n           4     0.1277    0.0820    0.0999      1000\n           5     0.1692    0.1890    0.1786      1000\n           6     0.2845    0.5690    0.3793      1000\n           7     0.1967    0.1070    0.1386      1000\n           8     0.2980    0.4070    0.3440      1000\n           9     0.2689    0.3300    0.2964      1000\n\n    accuracy                         0.2216     10000\n   macro avg     0.2077    0.2216    0.2027     10000\nweighted avg     0.2077    0.2216    0.2027     10000\n'

In [ ]:
#stacking ensemble
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
model_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].outputs[0])
model_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].outputs[0])
model_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].outputs[0])
model_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].outputs[0])
model_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].outputs[0])
model_1.trainable = False
model_2.trainable = False
model_3.trainable = False
model_4.trainable = False
model_5.trainable = False
inputs = keras.Input(shape = input_size)
prediction_1 = model_1(inputs, training = False)
prediction_2 = model_2(inputs, training = False)
prediction_3 = model_3(inputs, training = False)
prediction_4 = model_4(inputs, training = False)
prediction_5 = model_5(inputs, training = False)
x = keras.layers.Concatenate(axis=-1)([prediction_1,prediction_2,prediction_3,prediction_4,prediction_5])
x = keras.layers.Dense(128, activation='relu', name='dense_100')(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='stacking.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
3/3 [==============================] - 6s 1s/step - loss: 2.2402 - accuracy: 0.1667 - val_loss: 2.2962 - val_accuracy: 0.0890
Epoch 2/50
3/3 [==============================] - 2s 979ms/step - loss: 2.1402 - accuracy: 0.2333 - val_loss: 2.2852 - val_accuracy: 0.1275
Epoch 3/50
3/3 [==============================] - 2s 956ms/step - loss: 2.0439 - accuracy: 0.5667 - val_loss: 2.2752 - val_accuracy: 0.1820
Epoch 4/50
3/3 [==============================] - 2s 988ms/step - loss: 1.9493 - accuracy: 0.7667 - val_loss: 2.2658 - val_accuracy: 0.1990
Epoch 5/50
3/3 [==============================] - 2s 988ms/step - loss: 1.8592 - accuracy: 0.8000 - val_loss: 2.2569 - val_accuracy: 0.2065
Epoch 6/50
3/3 [==============================] - 2s 996ms/step - loss: 1.7716 - accuracy: 0.8667 - val_loss: 2.2490 - val_accuracy: 0.2120
Epoch 7/50
3/3 [==============================] - 2s 977ms/step - loss: 1.6840 - accuracy: 0.9000 - val_loss: 2.2416 - val_accuracy: 0.2170
Epoch 8/50
3/3 [=======

In [ ]:
#finetuning
model_1.trainable = True
model_2.trainable = True
model_3.trainable = True
model_4.trainable = True
model_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='stacking.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
3/3 [==============================] - 9s 2s/step - loss: 0.1707 - accuracy: 1.0000 - val_loss: 2.4972 - val_accuracy: 0.2255
Epoch 2/20
3/3 [==============================] - 2s 933ms/step - loss: 0.1693 - accuracy: 1.0000 - val_loss: 2.5030 - val_accuracy: 0.2245
Epoch 3/20
3/3 [==============================] - 2s 927ms/step - loss: 0.1681 - accuracy: 1.0000 - val_loss: 2.5085 - val_accuracy: 0.2245
Epoch 4/20
3/3 [==============================] - 2s 945ms/step - loss: 0.1675 - accuracy: 1.0000 - val_loss: 2.5128 - val_accuracy: 0.2235
Epoch 5/20
3/3 [==============================] - 2s 935ms/step - loss: 0.1670 - accuracy: 1.0000 - val_loss: 2.5164 - val_accuracy: 0.2245
Epoch 6/20
3/3 [==============================] - 2s 934ms/step - loss: 0.1664 - accuracy: 1.0000 - val_loss: 2.5199 - val_accuracy: 0.2250
Epoch 7/20
3/3 [==============================] - 2s 946ms/step - loss: 0.1660 - accuracy: 1.0000 - val_loss: 2.5231 - val_accuracy: 0.2255
Epoch 8/20
3/3 [=======

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.2395    0.1480    0.1829      1000\n           1     0.2678    0.2030    0.2309      1000\n           2     0.1526    0.1270    0.1386      1000\n           3     0.1865    0.1770    0.1816      1000\n           4     0.1195    0.0810    0.0965      1000\n           5     0.1912    0.1390    0.1610      1000\n           6     0.2815    0.5680    0.3764      1000\n           7     0.1908    0.2310    0.2090      1000\n           8     0.3709    0.3690    0.3699      1000\n           9     0.2710    0.3290    0.2972      1000\n\n    accuracy                         0.2372     10000\n   macro avg     0.2271    0.2372    0.2244     10000\nweighted avg     0.2271    0.2372    0.2244     10000\n'